### Retriever

리트리버는 비정형 쿼리가 주어지면 문서를 반환하는 인터페이스입니다. 벡터 저장소보다 더 일반적입니다. 리트리버는 문서를 저장할 필요 없이 단지 반환(또는 검색)만 할 수 있습니다. - *Langchain Document*

쉽게 말해, Retriever는 검색을 쉽게 할 수 있도록 구성된 모듈입니다. 이를 통해 손쉽게 문서를 검색할 수 있도록 하고 이를 기반으로 LLM과 대화할 수 있도록 합니다.

In [ ]:
!pip install -q chromadb tiktoken transformers sentence_transformers openai langchain pypdf

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import openai
os.environ["OPENAI_API_KEY"] = ## API KEY ##

In [ ]:
import tiktoken

tokenizer = tiktoken.get_encoding("cl100k_base")

def tiktoken_len(text):
    tokens = tokenizer.encode(text)
    return len(tokens)

In [ ]:
!pip install -q langchain_community

In [ ]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import PyPDFLoader

In [ ]:
loader = PyPDFLoader("/content/drive/MyDrive/2025 외부 LLM/자료/[이슈리포트 2022-2호] 혁신성장 정책금융 동향.pdf")
pages = loader.load_and_split()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50, length_function = tiktoken_len)
texts = text_splitter.split_documents(pages)

from langchain.embeddings import HuggingFaceEmbeddings

model_name = "jhgan/ko-sbert-nli"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

docsearch = Chroma.from_documents(texts, hf)

<ipython-input-7-4b27f93c590d>:12: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  hf = HuggingFaceEmbeddings(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

openai = ChatOpenAI(model_name="gpt-3.5-turbo",
                    streaming=True, callbacks=[StreamingStdOutCallbackHandler()], #스트리밍으로 답변
                    temperature = 0) # 0 이면 일관된 답변을 함 (다양성을 낮게)

# 리트리벌 QA체인 선언
qa = RetrievalQA.from_chain_type(llm = openai,
                                 chain_type = "stuff",  # stuff 체인타입 : 청크사이즈를 500으로 설정했으므로 3개를 참고해도 1500, 따라서 max 청크 사이즈 4000(GPT3.5)을 넘지 않음, 청크가 길어질것 같으면 map reduce, refine사용
                                 retriever = docsearch.as_retriever( # 크로마객체의 docsearch를 검색기로 사용
                                    search_type="mmr", # MMR(Maximal Marginal Relevance) 방식으로 검색하여 관련 문서를 선택함. 중복을 줄이고 다양한 관점을 반영할 수 있음
                                    search_kwargs={'k':3, 'fetch_k': 10}), # fetch_k는 연관된 총 10개중에서 조합, k는 총 3개만 넘긴다는 의미
                                 return_source_documents = True) #소스정보를 제공

query = "혁신성장 정책금융에 대해서 설명해줘"
result = qa(query)

혁신성장 정책금융은 기업의 성장과 혁신을 촉진하기 위해 정부나 정부가 지원하는 기관이 제공하는 금융 지원 제도를 말합니다. 이를 통해 새로운 기술이나 아이디어를 가진 기업들이 성장할 수 있도록 자금을 지원하고, 혁신적인 사업 모델을 구축하고 확장할 수 있도록 돕습니다. 혁신성장 정책금융은 혁신적인 산업을 육성하고 국가의 경제 성장을 촉진하는 데 중요한 역할을 합니다. 이를 통해 새로운 산업 분야의 경쟁력을 향상시키고 미래 산업에 대한 지속적인 투자를 유도하는데 기여합니다.혁신성장 정책금융은 기업의 성장과 혁신을 촉진하기 위해 정부나 정부가 지원하는 기관이 제공하는 금융 지원 제도를 말합니다. 이를 통해 새로운 기술이나 아이디어를 가진 기업들이 성장할 수 있도록 자금을 지원하고, 혁신적인 사업 모델을 구축하고 확장할 수 있도록 돕습니다. 또한, 혁신성장산업 기업발굴 가이드라인의 내실화·최신화에 노력을 기해야 함과 미래 성장성은 유망하나 단기 수익 창출이 어려운 산업의 지원 강화를 위해 정책금융 뿐만 아니라 민관주도의 역동적 금융으로 혁신성장 금융지원 영역을 확대할 필요가 있음을 고려하여 혁신성장 정책금융의 중요성을 강조할 수 있습니다.혁신성장 정책금융은 기업의 성장과 혁신을 촉진하기 위해 정부나 정부가 지원하는 기관이 제공하는 금융 지원 제도를 말합니다. 이를 통해 새로운 기술이나 아이디어를 가진 기업들이 성장할 수 있도록 자금을 지원하고, 혁신적인 사업 모델을 구축하고 확장할 수 있도록 돕습니다. 특히 ICT 산업을 중심으로 혁신성장 정책금융이 활발히 이루어지고 있으며, 이를 통해 디지털 기술을 적극적으로 활용하는 기업들이 성장할 수 있도록 지원하고 있습니다. 또한, 혁신성장산업 기업발굴 가이드라인의 내실화·최신화에 노력을 기해야 함과 미래 성장성은 유망하나 단기 수익 창출이 어려운 산업의 지원 강화를 위해 정책금융 뿐만 아니라 민관주도의 역동적 금융으로 혁신성장 금융지원 영역을 확대할 필요가 있음을 고려하여 혁신성장 정책금융의 중요성을 강조할 수 있습니다.

In [ ]:
result

{'query': '혁신성장 정책금융에 대해서 설명해줘',
 'result': '혁신성장 정책금융은 기업이 혁신적인 아이디어나 기술을 발전시키고 성장할 수 있도록 정부나 정책금융기관이 제공하는 금융 지원을 말합니다. 이를 통해 새로운 산업이나 기술이 발전하고 경제 성장을 이끌어내는 데 도움을 줍니다. 이러한 혁신성장 정책금융은 기업 발굴, 자금 지원, 그리고 산업 육성을 위한 다양한 제도와 지침에 따라 운영됩니다. 위의 문맥에서는 ICT 산업을 중심으로 혁신성장 정책금융의 동향이 다루어지고 있습니다.',
 'source_documents': [Document(metadata={'author': 'kmd kdy', 'creationdate': '2022-07-29T09:03:16+09:00', 'creator': 'Hancom PDF 1.3.0.538', 'moddate': '2022-07-29T09:03:16+09:00', 'page': 2, 'page_label': '3', 'pdfversion': '1.4', 'producer': 'Hancom PDF 1.3.0.538', 'source': '/content/drive/MyDrive/2025 외부 LLM/자료/[이슈리포트 2022-2호] 혁신성장 정책금융 동향.pdf', 'total_pages': 18}, page_content='1. 들어가며▶혁신성장 정책금융기관은 건강한 혁신산업 생태계를 조성하기 위해 기업 성장에 필요한 자금을 지원하는 혁신성장 정책금융 제도를 시행하고 있음￮혁신성장 정책금융기관은 혁신성장에 대한 정의를 구체화한 정책금융 가이드라인*에 따라 혁신성장 산업육성을 위한 정책금융 업무를 추진 중임       * 혁신성장 기업발굴 및 금융지원을 위해 활용하는 기준으로, ‘9대 테마-46개 분야-296개 품목’으로 구성▶혁신성장 정책금융 제도 시행 이후 공급 규모가 매년 증가하는 등, 미래 혁신성장 분야의 글로벌 경쟁력 확보를 위한 금융지원이 지